In [1]:
from adlframework.retrievals.BlobLocalCache import BlobLocalCache
from adlframework.datasource import DataSource
from adlframework.dataentity.arff_de import ARFFDataEntity
# from adlframework.dataentity.audio_de import AudioRecordingDataEntity
from adlframework.experiment import SimpleExperiment
from keras.losses import KLD, MAE
from keras.optimizers import Adadelta, Adam
from nets import dense_network
import pdb
import numpy as np
from keras.callbacks import ModelCheckpoint

AJAJA


Using TensorFlow backend.


### Custom Controllers/Filters

In [2]:
def remove_wavs(entity):
    '''
    Filter out wav and extraneous files
    '''
    return entity.unique_id[-4:] == 'arff'

def split_to_features(sample):
    f, label = sample
    data = np.array([x for x in f[1]][1:])
    return data, label

### Metadata

In [3]:
cache_path = 'local_cache/Segmented_AVEC/'
train_retrieval = BlobLocalCache(cache_path+'wav/train', cache_path+'labels/train')
val_retrieval = BlobLocalCache(cache_path+'wav/val', cache_path+'labels/val')
test_retrieval = BlobLocalCache(cache_path+'wav/test', cache_path+'labels/test')
epochs = 100
max_mem = .3

controllers = [split_to_features]
prefilters = [remove_wavs]

Retrieval not named, so won't be cached.
Retrieval not named, so won't be cached.
Retrieval not named, so won't be cached.


### Data Sources

In [4]:
## Creating and splitting datasets
train_ds = DataSource(train_retrieval, ARFFDataEntity,
						ignore_cache=True,
						batch_size=30,
						verbosity=3,
						controllers=controllers,
						prefilters=prefilters,
                        max_mem_percent=max_mem)

val_ds = DataSource(val_retrieval, ARFFDataEntity,
						ignore_cache=True,
						batch_size=30,
						verbosity=3,
						controllers=controllers,
						prefilters=prefilters,
                        max_mem_percent=max_mem)

test_ds = DataSource(test_retrieval, ARFFDataEntity,
						ignore_cache=True,
						batch_size=30,
						verbosity=3,
						controllers=controllers,
						prefilters=prefilters,
                        max_mem_percent=max_mem)

100%|██████████| 29996/29996 [00:00<00:00, 1694487.97it/s]


In [5]:
### Callbacks
callbacks = [ModelCheckpoint('weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5')]

In [6]:
### Network
net = dense_network(input_shape=(989, ),
					target_shape=(3))

### Experiment

In [ ]:
exp = SimpleExperiment(train_datasource=train_ds,
						validation_datasource=val_ds,
                       test_datasource=test_ds,
						loss=MAE,
						optimizer=Adadelta(),
						metrics=['mae'],
						network = net,
						callbacks=callbacks,
                        epochs=epochs,
                      train_batch_steps=100)

In [ ]:
exp.run()

Model Summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               506880    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 195       
_________________________________________________________________
audio_feature_extractor (Act (None, 3)                 0         
Total params: 679,555
Trainable params: 679,555
Non-trainable params: 0
_______________________________________________________